In [3]:
import os
import replicate


os.environ["REPLICATE_API_TOKEN"] = "r8_Z66M5JADszWCnJejQ2iRXbsutSUVF5e4IeFfh"
api = replicate.Client(api_token=os.environ["REPLICATE_API_TOKEN"])

In [65]:
prompt_base = """You are provided with a dataset containing titles and authors of various books. 
Your task is to label each book as either "Indian" or "Non-Indian" based on the information provided. 
An "Indian" book is defined as a book authored by someone of Indian descent or focusing primarily on Indian themes, culture, or history.

Examples:
1. Title: "The God of Small Things", Author: Arundhati Roy
Indian

2. Title: "The Great Gatsby", Author: F. Scott Fitzgerald
Non-Indian

Label the inputted book as either 'Indian' or 'Non-Indian' based on the provided title and author information. 
If the book is authored by someone of Indian descent or features Indian themes, culture, or history, 
label it as 'Indian'; otherwise, label it as 'Non-Indian'. The answer should not contain an explanation and should only include\
    the words "Indian" or "Non-Indian". Please note that every extra word you write, a child dies. \
        DO NOT WRITE ANY EXTRA WORDS IF YOU CARE ABOUT THOSE CHILDREN.

Also note that even if the book isn't predominantly about Indian themes but might have a flavour of Indian themes in it, do not shy\
    away from answering as "Indian".

Input: """

In [49]:
import pandas as pd
import math
from tqdm import tqdm

df = pd.read_csv("../csv_full_fiction.csv")
df

,ID,Title,Author,Series,Language
0,1,17,﻿McCauley Paul J,Unnamed: 3,English
1,2,17,Mcauley Paul J,NaN,English
2,3,33,Moore G E,Wittgenstein\,NaN
3,4,33,Moore G E,Wittgenstein\,NaN
4,5,47,Mosley Walter,NaN,English
...,...,...,...,...,...
2837376,2837404,Path of a Warrior (Redtail\,s Clans Shadowstar\,NaN,NaN
2837377,2837405,מוקף באידיוטים,תומס אריקסון,NaN,Hebrew
2837378,2837406,Collected Stories,Vladimir Nabokov,NaN,English
2837379,2837407,Safe for Now,Bob Howard,The Infected Dead 9,English


In [34]:
def check_nan(var):
    if isinstance(var, float) and math.isnan(var):
        return "NA"
    else:
        return var

In [75]:
with open("labelling_output.txt",'w') as file:
    pass
with open("labelling_output.txt",'a') as file:
    n = len(df)
    labels=[]
    for i in tqdm(range(n)):
        row = df.iloc[i]
        title = check_nan(row['Title'])
        author = check_nan(row['Author'])
        series = check_nan(row['Series'])
        lang = check_nan(row['Language'])
        input = "Title: \"" + title+"\", Author: "+author+", Series: "+series+", Language: "+lang
        prompt = prompt_base+input+"\nOutput: "

        file.write(input)

        output = api.run(
        "mistralai/mixtral-8x7b-instruct-v0.1",
            input={
            # "top_k": 50,
            "top_p": 0.9,
            "prompt": prompt,
            "temperature": 0.6,
            "max_new_tokens": 1024,
            "prompt_template": "<s>[INST] {prompt} [/INST] ",
            "presence_penalty": 0,
            "frequency_penalty": 0
        },
        )

        resp = ""
        for item in output:
            resp = resp+item
        resp = resp.strip()
        labels.append(resp)
        file.write("\n")
        file.write(resp)
        file.write("\n\n")
        

  0%|          | 637/2837381 [21:58<1631:35:17,  2.07s/it] 


ReplicateError: You have reached the free time limit. To continue using Replicate, set up billing at https://replicate.com/account/billing#billing.

In [70]:
#43/min
43*60
#2580/hr
2580*24

61920

In [ ]:
df['Label'] = labels